In [1]:
!pwd

/data5/galaxy/project/methyl_m6a


#### proprocess data

In [2]:
import os
import shutil
import glob
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from itertools import permutations
from multiprocessing import Pool

In [7]:
bw_list = glob.glob("data/*.bigwig")
print(bw_list)
for bw in bw_list:
    wiggle_file = "data/wig_format/%s" % (os.path.basename(bw).split(".")[0])
    print(wiggle_file)
    os.system("bigWigToWig %s %s" % (bw, wiggle_file))

['data/E081_RRBS_FractionalMethylation.bigwig', 'data/E088_RRBS_FractionalMethylation.bigwig', 'data/E086_RRBS_FractionalMethylation.bigwig', 'data/E083_RRBS_ReadCoverage.bigwig', 'data/E081_RRBS_ReadCoverage.bigwig', 'data/E083_RRBS_FractionalMethylation.bigwig', 'data/E088_RRBS_ReadCoverage.bigwig', 'data/E086_RRBS_ReadCoverage.bigwig']
data/wig_format/E081_RRBS_FractionalMethylation


0

data/wig_format/E088_RRBS_FractionalMethylation


0

data/wig_format/E086_RRBS_FractionalMethylation


...

0

data/wig_format/E088_RRBS_ReadCoverage


0

data/wig_format/E086_RRBS_ReadCoverage


0

In [1]:
# cov_list, frac_list = glob.glob("data/wig_format/*ReadCoverage"), glob.glob("data/wig_format/*FractionalMethylation")
# print(len(cov_list))
# def get_dict(in_list):
#     result_dict = {}
#     for i_file in in_list:
#         prefix = os.path.basename(i_file).split("_RRBS")[0]
#         result_dict[prefix] = i_file
#     return result_dict
# cov_dict, frac_dict = get_dict(cov_list), get_dict(frac_list)
# print(cov_dict["E086"], frac_dict["E086"])

In [21]:
# use end as position;  term the coverge file number as the total number;
# def format_into_methylCount(prefix, cov_file, frac_file):
#     df_cov = pd.read_table(cov_file, sep="\t", header=None, comment="#", names=["a", "b", "c", "number"])
#     df_frac = pd.read_table(frac_file, sep="\t", header=None, comment="#", names=["a", "b", "c", "frac"])
#     df = df_cov.merge(df_frac).drop_duplicates()
#     df["position"], df["strand"], df["type"] = df["c"], "+", "CpG"
#     # df["name"] = df["a"].astype(str) + ":" + df["position"].astype(str)
#     df = df[["a", "position", "strand", "type", "frac", "number"]]
#     result_file = "data/methylcount_format/%s.meth" % prefix
#     print(df.head())
#     df.to_csv(result_file, sep="\t", header=None, index=False)
# prefix_list = cov_dict.keys()
# for pre in prefix_list:
#     cov_file, frac_file = cov_dict[pre], frac_dict[pre]
#     format_into_methylCount(pre, cov_file, frac_file)

In [20]:
# merge seems didn't work
# def merge_paired_CpG_site(meth_file):
#     result_file = "data/merged_meth/%s" % os.path.basename(meth_file)
#     os.system("symmetric-cpgs -o %s %s" % (result_file, meth_file))
# meth_list = glob.glob("data/methylcount_format/*.meth")
# for meth_file in meth_list:
#     merge_paired_CpG_site(meth_file)

In [47]:
# def get_up_down_peak(df, col_one, col_two):
#     df = df.copy()[["chr", "start", "end", col_one, col_two]]
#     df["2vs1"] = df[col_two] / df[col_one]
#     df_up, df_down = df[df["2vs1"] > 2], df[df["2vs1"] < 0.5]
#     df_up, df_down = df_up[["chr", "start", "end"]], df_down[["chr", "start", "end"]]
# #     print(df_up.head())
# #     print(df_down.head())
#     return df_up, df_down

# m6a_level = "/data5/xlj/new_tissue_m6A/Macs2/Filter/Merge-Dup/Ratio/Total.Ratio"
# df = pd.read_table(m6a_level, sep="\t")
# df["chr"], df["start"], df["end"] = df.peak.str.split(":").str[0], df.peak.str.split(":").str[1].str.split("-").str[0], df.peak.str.split("-").str[1]
# # df.head()
# df_up, df_down = get_up_down_peak(df, "brain", "heart")

# df_up["start"] = df_up["start"].astype(int)
# df_down["start"] = df_down["start"].astype(int)
# df_up = df_up.sort_values(["chr", "start"])
# df_down = df_down.sort_values(["chr", "start"])
# df_up.to_csv("/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/Up_m6a.bed", sep="\t", header=None, index=False)
# df_down.to_csv("/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/Down_m6a.bed", sep="\t", header=None, index=False)

In [31]:
def format_raw_bedgraph():
    os.chdir("/data5/galaxy/project/methyl_m6a/data/wig_format")
    frac_list = glob.glob("*FractionalMethylation")
    for frac in frac_list:
        df = pd.read_table(frac, sep="\t", comment="#", header=None, names=["chr", "start", "end", "score"])
        df = df[df.index % 2 == 0]
        prefix, chain_file = os.path.basename(frac).split("_")[0], "/data/database/chain_file/hg19ToHg38.over.chain"
        df["name"] = df["chr"] + ":" + df["start"].astype(str) + "-" + df["end"].astype(str)
        df["strand"] = "."
        df_bed = df[["chr", "start", "end", "name", "score", "strand"]]
        raw_result, final_result = ("raw_%s.bed" % prefix), ("%s.bed" % prefix)
        df_bed.to_csv(raw_result, sep="\t", header=None, index=False)
        os.system("liftOver %s %s %s unmapped.bed" % (raw_result, chain_file, final_result))
        os.remove(raw_result)
        print("format_raw_bedgraph has done!")
#
format_raw_bedgraph()

format_raw_bedgraph has done!
format_raw_bedgraph has done!
format_raw_bedgraph has done!
format_raw_bedgraph has done!


In [35]:
def calculate_promoter_methylation_level():
    os.chdir("/data5/galaxy/project/methyl_m6a/data/roimethstat")
    promoter_bed = "/data5/galaxy/project/data/promoter/human/2k_100/genes_promoters.bed"
    bed_list = glob.glob("/data5/galaxy/project/methyl_m6a/data/wig_format/*.bed")
    for formatted_bed in bed_list:
        print(formatted_bed)
        prefix, raw_result = os.path.basename(formatted_bed).split(".bed")[0], "roimathstat_%s" % os.path.basename(formatted_bed)
        os.system("roimethstat -o %s %s %s" % (raw_result, promoter_bed, formatted_bed))
#         df = pd.read_table(raw_result, header=None, names=["chr", "a", "b", "name", prefix, "strand"])
#         df["start"], df["end"] = df["name"].str.split(":").str[-2], df["name"].str.split(":").str[-1]
#         df["name"] = df["name"].str.split(":").str[0]
#         df = df[["chr", "start", "end", "name", prefix, "strand"]]
#         result_bed = "promoter_%s" % os.path.basename(formatted_bed)
#         df.to_csv(result_bed, sep="\t", header=None, index=False)
        print("calculate_promoter_methylation_level has done!")
calculate_promoter_methylation_level()

/data5/galaxy/project/methyl_m6a/data/wig_format/E088.bed
calculate_promoter_methylation_level has done!
/data5/galaxy/project/methyl_m6a/data/wig_format/E083.bed
calculate_promoter_methylation_level has done!
/data5/galaxy/project/methyl_m6a/data/wig_format/E081.bed
calculate_promoter_methylation_level has done!
/data5/galaxy/project/methyl_m6a/data/wig_format/E086.bed
calculate_promoter_methylation_level has done!


#### diff m6a

In [77]:
base_dir = "/data5/galaxy/project/unique_bam/"
ip_dir, input_dir = "%s/ip/rename_bam" % base_dir, "%s/input/rename_bam" % base_dir
def call_diff_peak(tissue_prefix):
    res_dir = "/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2/%s" % tissue_prefix
    if not os.path.exists(res_dir):
          os.makedirs(res_dir)
    tr_bam = " ".join(glob.glob("%s/%s*.bam" % (ip_dir, tissue_prefix)))
    # print(tr_bam)
    con_bam = " ".join(glob.glob("%s/%s*.bam" % (input_dir, tissue_prefix)))
#     print(con_bam)
    os.system("macs2 callpeak -B -t %s -c %s --outdir %s -q 0.05 --nomodel -n %s -g hs" % (tr_bam, con_bam, res_dir, tissue_prefix))
tissue_list = list(set([os.path.basename(x).split("_")[0] for x in glob.glob("%s/*.bam" % ip_dir)]))
for tissue_prefix in tissue_list:
    print(tissue_prefix)
    call_diff_peak(tissue_prefix)

liver
/data5/galaxy/project/unique_bam//ip/rename_bam/liver_3.bam /data5/galaxy/project/unique_bam//ip/rename_bam/liver_2.bam /data5/galaxy/project/unique_bam//ip/rename_bam/liver_1.bam
/data5/galaxy/project/unique_bam//input/rename_bam/liver_1_L6.bam /data5/galaxy/project/unique_bam//input/rename_bam/liver_3_L6.bam /data5/galaxy/project/unique_bam//input/rename_bam/liver_2_L7.bam
lung
/data5/galaxy/project/unique_bam//ip/rename_bam/lung_5.bam /data5/galaxy/project/unique_bam//ip/rename_bam/lung_4.bam
/data5/galaxy/project/unique_bam//input/rename_bam/lung_5_L7.bam /data5/galaxy/project/unique_bam//input/rename_bam/lung_4_L7.bam
mus
/data5/galaxy/project/unique_bam//ip/rename_bam/mus_5.bam /data5/galaxy/project/unique_bam//ip/rename_bam/mus_4.bam
/data5/galaxy/project/unique_bam//input/rename_bam/mus_5_L7.bam /data5/galaxy/project/unique_bam//input/rename_bam/mus_4_L6.bam
pla
/data5/galaxy/project/unique_bam//ip/rename_bam/pla_6.bam /data5/galaxy/project/unique_bam//ip/rename_bam/pla_2

In [3]:
base_dir = "/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2"
tissue_list = [x.split("/")[-2] for x in glob.glob("%s/*/*.xls" % base_dir)]
pair_value_list = list(permutations(tissue_list, 2))   # combinations
    
def get_effective_seq_depth(tissue_prefix):
    treat_bdg = "%s/%s/%s_treat_pileup.bdg" % (base_dir, tissue_prefix, tissue_prefix)
    control_bdg = "%s/%s/%s_control_lambda.bdg" % (base_dir, tissue_prefix, tissue_prefix)
    xls = "%s/%s/%s_peaks.xls" % (base_dir, tissue_prefix, tissue_prefix)
    return_str = os.popen('egrep "tags after filtering in treatment|tags after filtering in control" %s' % xls)
    line_list, number_list = return_str.read().strip().split("\n"), []
    for line in line_list:
        print(line + "a")
        number_list.append(float(line.split(":")[1].strip()))
    final_num = min(number_list)
    return treat_bdg, control_bdg, final_num

def process_each_pair(pair_list):
    tissue_1, tissue_2 = pair_list[0], pair_list[1]
    print(tissue_1, tissue_2)
    result_dir = "/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2_bdgdiff/%s_vs_%s" % (tissue_1, tissue_2)
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    treat_bdg_1, control_bdg_1, final_num_1 = get_effective_seq_depth(tissue_1)
    treat_bdg_2, control_bdg_2, final_num_2 = get_effective_seq_depth(tissue_2)
    os.system("macs2 bdgdiff --t1 %s --c1 %s --t2 %s --c2 %s --d1 %f --d2 %f --outdir %s --o-prefix 'diff_peak'" % 
              (treat_bdg_1, control_bdg_1, treat_bdg_2, control_bdg_2, final_num_1, final_num_2, result_dir))

pool = Pool(processes=5)
for pair_list in pair_value_list:
    pool.apply_async(process_each_pair, (pair_list, ))
pool.close()
pool.join()

heart sto


heart lung


heart mus


heart pla


heart liver


...

# tags after filtering in treatment: 16858672a


# tags after filtering in control: 33858979a
# tags after filtering in control: 40349670a
# tags after filtering in treatment: 32181303a
# tags after filtering in control: 52008099a


# tags after filtering in treatment: 16997942a
# tags after filtering in control: 41728116a


heart brain
# tags after filtering in treatment: 19639808a
# tags after filtering in control: 47661421a
# tags after filtering in treatment: 43529061a
# tags after filtering in control: 64428658a
heart kid
# tags after filtering in treatment: 19639808a
# tags after filtering in control: 47661421a
# tags after filtering in treatment: 40433534a
# tags after filtering in control: 63313205a
sto heart
# tags after filtering in treatment: 12332131a
# tags after filtering in control: 41033398a
# tags after filtering in treatment: 19639808a
# tags after filtering in control: 47661421a
sto lung
# tags after filtering in treatment: 12332131a
# tags after filtering in control: 41033398a
# tags after filtering in treatment: 16858672a
# tags after filtering in control: 40349670a
sto mus
# tags after filtering in treatment: 12332131a
# tags after filtering in control: 41033398a
# tags after filtering in treatment: 20643431a
# tags after filtering in control: 33858979a
sto pla
# tags after filtering 

# tags after filtering in control: 64428658a
# tags after filtering in treatment: 16997942a
# tags after filtering in control: 41728116a
brain kid
# tags after filtering in treatment: 43529061a
# tags after filtering in control: 64428658a
# tags after filtering in treatment: 40433534a
# tags after filtering in control: 63313205a
kid heart
# tags after filtering in treatment: 40433534a
# tags after filtering in control: 63313205a
# tags after filtering in treatment: 19639808a
# tags after filtering in control: 47661421a
kid sto
# tags after filtering in treatment: 40433534a
# tags after filtering in control: 63313205a
# tags after filtering in treatment: 12332131a
# tags after filtering in control: 41033398a
kid lung
# tags after filtering in treatment: 40433534a
# tags after filtering in control: 63313205a
# tags after filtering in treatment: 16858672a
# tags after filtering in control: 40349670a
kid mus
# tags after filtering in treatment: 40433534a
# tags after filtering in control: 

In [4]:
# base_dir = "/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2_bdgdiff/"  # heart_vs_brain
# !python /data5/galaxy/shell_dir/2018_3_17/chip_seq_pipeline/short_reads/04_annotate_peaks.py

In [9]:
os.chdir("/data5/galaxy/project/expression/DESeq2")
subdir_name_list = [os.path.basename(x).split(".csv")[0] for x in glob.glob("*.csv")]
for sub_dir in subdir_name_list:
    print(sub_dir)
    source_dir = os.path.join("/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2_bdgdiff/permutation/", sub_dir)
    target_dir = os.path.join("/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2_bdgdiff/combination/", sub_dir)
    shutil.copytree(source_dir, target_dir)

mus_vs_pla


'/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2_bdgdiff/combination/mus_vs_pla'

heart_vs_sto


'/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2_bdgdiff/combination/heart_vs_sto'

lung_vs_liver


...

'/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2_bdgdiff/combination/lung_vs_heart'

lung_vs_sto


'/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2_bdgdiff/combination/lung_vs_sto'

mus_vs_brain


'/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2_bdgdiff/combination/mus_vs_brain'

In [11]:
m6a_dir = "/data5/galaxy/project/methyl_m6a/data/diff_m6a_peak/macs2_bdgdiff/combination/"
expre_dir = "/data5/galaxy/project/expression/DESeq2/"
# subdir_list = list(set([x.split("/")[-2] for x in glob.glob("%s/*/diff_peak_c3.0_common.bed" % m6a_dir)]))
# methylation level only have these tissue data.
subdir_list = ["heart_vs_brain", "kid_vs_brain", "lung_vs_brain", "heart_vs_kid", "lung_vs_heart", "lung_vs_kid"]
for sub_dir in subdir_list:
    print(sub_dir)
    df_methyl = foldchange_of_methylation(sub_dir)
    df_up, df_down, df_remain = class_genes_by_diffPeak(df_methyl, os.path.join(m6a_dir, sub_dir))
    df_gene_up, df_gene_down = get_diff_expression_genes(os.path.join(expre_dir, "%s.csv" % sub_dir))
    check_intersect_between_up_down(df_up, df_down, df_remain)
    df_up, df_up_unchange = filter_each_row_by_expression(df_up, df_gene_up, df_gene_down)
    df_down, df_down_unchange = filter_each_row_by_expression(df_down, df_gene_up, df_gene_down)
    df_remain, df_remain_unchange = filter_each_row_by_expression(df_remain, df_gene_up, df_gene_down)
    boxplot(df_up, df_up_unchange, df_down, df_down_unchange, df_remain, df_remain_unchange, sub_dir)
#     prepare_plot_data(sub_dir, df_up, df_down, df_remain)

heart_vs_brain
0.17827542509660232 0.14620232429314814
11358
5468 5890
6931 2967 5688
6931 2967 5688
6931 1307 4192
2967 921 1196
5688 635 4386
0.7399539626713317 -0.4012389067377619 0.39178322209562927
kid_vs_brain
0.1877232534897525 0.14620232429314814
10154
5562 4592
7680 3464 4881
7680 3464 4881
7680 1160 5024
3464 1158 1579
4881 395 4100
0.8508135854474768 -0.2096513636182674 0.32917379082827236
lung_vs_brain
0.18365661506835906 0.14620232429314814
10872
5569 5303
7736 2300 5517
7736 2300 5517
7736 1339 4867
2300 818 825
5517 633 4251
0.8960536065169726 -0.19235058169826766 0.36398260388349735
heart_vs_kid
0.17827542509660232 0.1877232534897525
7512
3100 4412
4927 4196 6834
4927 4196 6834
4927 848 3386
4196 603 2734
6834 383 5948
0.31803335046747505 -0.6408484864062493 0.028625855161572816
lung_vs_heart
0.18365661506835906 0.17827542509660232
6817
3758 3059
5348 2721 7473
5348 2721 7473
5348 740 3784
2721 658 1483
7473 432 6606
0.7871010177573928 -0.36624927696581067 0.12095590215

In [68]:
def prepare_plot_data(sub_dir, df_up, df_down, df_remain):
    result_file = "/data5/galaxy/project/methyl_m6a/data/plot_data/%s.txt" % sub_dir
    df_up["m6a_type"] = "up"
    df_down["m6a_type"] = "down"
    df_remain["m6a_type"] = "remain"
    df = pd.concat([df_up, df_down, df_remain], sort=True)
    df.to_csv(result_file, sep="\t", index=False)

In [3]:
# 81--brain; 83--heart; 86--kid; 88--lung;
# ["heart_vs_brain", "kid_vs_brain", "lung_vs_brain", "heart_vs_kid", "lung_vs_heart", "lung_vs_kid"]
def foldchange_of_methylation(sub_dir):
    os.chdir("/data5/galaxy/project/methyl_m6a/data/roimethstat/")
    map_dict = {"brain": "E081", "heart": "E083", "kid": "E086", "lung": "E088"}
    tissue_1, tissue_2 = sub_dir.split("_")[0], sub_dir.split("_")[-1]
    try:
        num_1, num_2 = map_dict[tissue_1], map_dict[tissue_2]
    except KeyError:
        print("%s %s didn't have acoording DNA methylation data!" % (tissue_1, tissue_2))
    df_1 = pd.read_table("promoter_%s.bed" % num_1, header=None, names=["chr", "s", "e", "n", tissue_1, "strand"])
    df_1["name"] = df_1["n"].str.split(":").str[0]
    df_2 = pd.read_table("promoter_%s.bed" % num_2, header=None, names=["chr", "s", "e", "n", tissue_2, "strand"])
    df_2["name"] = df_2["n"].str.split(":").str[0]
    df_methyl = df_1.merge(df_2, on="name").dropna()
    df_methyl["logFC"] = np.log2(df_methyl[tissue_1].astype(float) + 0.01) - np.log2(df_methyl[tissue_2].astype(float) + 0.01)
    df_methyl = df_methyl[["name", "logFC"]]
    print(df_1[tissue_1].mean(), df_2[tissue_2].mean())
    return df_methyl

In [4]:
def class_genes_by_diffPeak(df_methylation, work_dir):
    os.chdir(work_dir)
    m6a_up = pd.read_table("diff_peak_c3.0_cond1_anno.txt", sep="\t", header=None, names=["name", "m6a_score"])
    m6a_down = pd.read_table("diff_peak_c3.0_cond2_anno.txt", sep="\t", header=None, names=["name", "m6a_score"])
    m6a_up = m6a_up.groupby(["name"]).mean().reset_index()
    m6a_down = m6a_down.groupby(["name"]).mean().reset_index()
    df_up = df_methylation.merge(m6a_up, how="right").dropna().drop_duplicates()
    df_down = df_methylation.merge(m6a_down, how="right").dropna().drop_duplicates()
    diff_genes = df_up["name"].tolist() + df_down["name"].tolist()
    remain_genes = [x for x in df_methylation["name"] if x not in diff_genes]
    df_remain = df_methylation[df_methylation["name"].isin(remain_genes)]
    df_remain = df_remain.drop(df_remain.index[len(df_remain) - 1]).replace([np.inf, -np.inf], np.nan).dropna().drop_duplicates()
    return df_up, df_down, df_remain

In [5]:
def format_gene_name(df):
    df["name"] = df["Unnamed: 0"].str.split(".").str[0]
    df = df.set_index(["name"])
    del df["Unnamed: 0"]
    return df

def get_diff_expression_genes(DESeq2_file):
    df = pd.read_csv(DESeq2_file)
    df = df[(abs(df.log2FoldChange) > 1) & (df.padj < 0.05)]
    print(len(df))
    df_up, df_down = df.copy()[df.log2FoldChange > 0], df.copy()[df.log2FoldChange < 0]
    print(len(df_up), len(df_down))
    return format_gene_name(df_up), format_gene_name(df_down)

In [6]:
def check_intersect_between_up_down(df_up, df_down, df_remain):
#     df_up, df_down, df_remain = correlation_between_m6a_and_methylation()
    print(len(df_up), len(df_down), len(df_remain))
    up_genes, down_genes, remain_genes = df_up["name"], df_down["name"], df_remain["name"]
    uniq_up = [name for name in up_genes if (name not in down_genes) and (name not in remain_genes)]
    uniq_down = [name for name in down_genes if (name not in up_genes) and (name not in remain_genes)]
    uniq_remain = [name for name in remain_genes if (name not in down_genes) and (name not in up_genes)]
    print(len(uniq_up), len(uniq_down), len(uniq_remain))

In [7]:
def filter_each_row_by_expression(df, df_gene_up, df_gene_down):
    df_pos, df_neg = df[df.logFC >0], df[df.logFC <0]
    up_genes, down_genes, total_genes = df_gene_up.index, df_gene_down.index, list(set(df["name"].tolist()))
    remain_genes = [x for x in total_genes if (x not in up_genes) and (x not in down_genes)]
    df_pos = df_pos[df_pos["name"].isin(down_genes)]
    df_neg = df_neg[df_neg["name"].isin(up_genes)]
    df_coordinate = pd.concat([df_pos, df_neg]).dropna()
    df_unchange = df[df["name"].isin(remain_genes)]
    print(len(df), len(df_coordinate), len(df_unchange))
    return df_coordinate, df_unchange

In [10]:
def boxplot(df_up, df_up_unchange, df_down, df_down_unchange, df_remain, df_remain_unchange, sub_dir):
    df_up_down = pd.DataFrame({"up": df_up["logFC"], "up_unchange": df_up_unchange["logFC"], 
                               "down": df_down["logFC"], "down_unchange": df_down_unchange["logFC"],
                               "remain": df_remain["logFC"], "remain_unchange": df_remain_unchange["logFC"]})
    g = sns.boxplot(data=df_up_down, order=["down", "down_unchange", "up", "up_unchange", "remain", "remain_unchange"], saturation=0.5, showfliers=False)
    g.set_ylabel("log2 fold change of methylation level", size = 14, color="black", alpha=1)
    plt.title(sub_dir)
    plt.savefig("/data5/galaxy/project/methyl_m6a/analysis_result_2/%s.pdf" % sub_dir)
    plt.close()
    print(df_up_down["down"].mean(), df_up_down["up"].mean(), df_up_down["remain"].mean())
    # return df_up_down